## Jupyter Notebook for checking the ribotree output metrics with fresh calculated metrics

Do Soon Kim, Stanford University
2021

**TO DO**:
- Make sure paths below are filled in with paths pointing to installations for the packages. 
*DSK had to specify path for arniefile, might be something with my miniconda or my new machine -- was not needed on my old computer.

**NEED**:
- arnie
- RiboGraphViz
- DegScore
- ribotree-mrna

In [1]:
#display all output in jupyter notebook
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#appending arnie to path
import sys, os
arnie_path = '/Users/dosoon/secstruct_software/'
sys.path.append(arnie_path)
# also setting the environment for arniefile
os.environ['ARNIEFILE'] = "/Users/dosoon/secstruct_software/arnie/arnie.arnie.txt"

#import mfe package from arnie, degscore, RGV, and calc_cai
sys.path.append('/Users/dosoon/secstruct_software/DegScore')
sys.path.append('/Users/dosoon/secstruct_software/RiboGraphViz/')
sys.path.append('/Users/dosoon/secstruct_software/ribotree-mrna/')
from arnie.mfe import mfe
from arnie.bpps import bpps
from DegScore import DegScore
from utils_features import calc_cai
from RiboGraphViz.RiboGraphViz import RiboGraphViz as RGV

#importing other packages for plotting
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
#reading in example output file, replace with desired file
data = pd.read_csv('orig.20210625-125737-23679.RUNNING_BEST.txt', sep='\t')

### Calculating fresh metrics from sequence and reported structure to double check Ribotree outputs

In [119]:
#quick function to take structure and return MLD
def return_MLD(struct):
    rg_mdl = RGV(struct)
    rg_mdl.run_structure_properties()
    return rg_mdl.MLD

#### Wrapped the recalculation steps into a function 

In [13]:
# input is a dataframe, returns a dataframe with new columns containing calculated metrics
def doublecheck_output(data):
    if ('full_sequence' in list(data.columns)):
        #updating dG(MFE) and structure with fresh calcs
        data[['MFE Structure_reproduce','dG(MFE)_reproduce']] = data.apply(lambda row: mfe(row['full_sequence'], package='eternafold',return_dG_MFE = True, linear=True), axis=1,result_type='expand')
        #and now also DegScore
        #note that the fresh calculated structure is used
        data['DegScore_reproduce'] = data.apply(lambda row: DegScore(sequence = row['full_sequence'], structure=row['MFE Structure_reproduce']).degscore, axis=1, result_type='expand')
        # AUP (Average Unpaired probability) calculation
        data['AUP_reproduce'] = data.apply(lambda row: np.mean(1 - np.sum(bpps(row['full_sequence'], package='eternafold', linear=True), axis=1)), axis=1)
        data['AUP_init14_reproduce'] = data.apply(lambda row: np.mean((1 - np.sum(bpps(row['full_sequence'], package='eternafold',linear=True), axis=1))[:14]), axis=1)
        #CAI calculation
        data['CAI_reproduce'] = data.apply(lambda row: calc_cai(row['CDS_sequence']), axis=1, result_type='expand')
        #and finally, MLD
        data['MLD_reproduce'] = data.apply(lambda row: return_MLD(row['MFE Structure_reproduce']), axis=1, result_type='expand')
    
    # if sequence does not have full_sequence column, then piece together the full sequence using
    # any specified UTR sequences and the CDS sequence
    elif ('sequence' in list(data.columns) and 'c3prime' in list(data.columns) and 'c5prime' in list(data.columns)):
        data[['MFE Structure_reproduce','dG(MFE)_reproduce']] = data.apply(lambda row: mfe(row['c5prime']+row['sequence']+row['c3prime'], package='eternafold',return_dG_MFE = True, linear=True), axis=1,result_type='expand')
        data['DegScore_reproduce'] = data.apply(lambda row: DegScore(sequence = row['c5prime']+row['sequence']+row['c3prime'], structure=row['MFE Structure_reproduce']).degscore, axis=1, result_type='expand')
        data['AUP_reproduce'] = data.apply(lambda row: np.mean(1 - np.sum(bpps(row['c5prime']+row['sequence']+row['c3prime'], package='eternafold', linear=True), axis=1)), axis=1)
        data['AUP_init14_reproduce'] = data.apply(lambda row: np.mean((1 - np.sum(bpps(row['c5prime']+row['sequence']+row['c3prime'], package='eternafold',linear=True), axis=1))[:14]), axis=1)
        data['CAI_reproduce'] = data.apply(lambda row: calc_cai(row['sequence']), axis=1, result_type='expand')
        data['MLD_reproduce'] = data.apply(lambda row: return_MLD(row['MFE Structure_reproduce']), axis=1, result_type='expand')
    
    else:
        raise Exception("Ribotree output file missing 'sequence', 'full_sequence', 'c5 prime', or 'c3 primer' columns.")
    return data

In [ ]:
# processing data
checked_data = doublecheck_output(data)

### Plotting

In [ ]:
sns.set_context('talk')
plt.figure(figsize=(20,4))

#list of metrics to examine:
metric_list = ['DegScore', 'dG(MFE)', 'AUP','AUP_init14', 'CAI','MLD']

num_plots = len(metric_list)
n_cols = num_plots
n_rows = 1

for i,metric in enumerate(metric_list):
    plt.subplot(n_rows, n_cols, i+1)
    
    #scatterplot to directly compare
    sns.scatterplot(data=checked_data, x=metric, y=metric+'_reproduce')
    plt.title(metric)
    #plotting x=y line
    xmin=checked_data[metric].min()
    xmax=checked_data[metric].max()
    perfect_fit = np.arange(xmin, xmax, step=(xmax-xmin)/100)
    sns.lineplot(x=perfect_fit, y=perfect_fit, linestyle='--', color='black', linewidth=1)
    plt.tight_layout()

### Printing out row by row to compare values (with decimal point precision)

In [160]:
for _, row in checked_data.iterrows():
    print('------')
    print('------')
    print("Checking for sequence: %s" %row['full_sequence'])
    #doing MFE structure separately since no 
    print("MFE Structure matches ribotree MFE Structure: %s" %str(row['MFE Structure']==row['MFE Structure_reproduce']))
    for metric in metric_list:
        print('.........')
        print("Ribotree-calculated %s: %s" %(metric,str(row[metric])))
        print("Calculated %s: %s" %(metric,str(row[metric+'_reproduce'])))
        

------
------
Checking for sequence: AUGGCUGUAUACCCUUACGACGUCCCUGAUUACGCCGGGUAUCCCUACGACGUCCCGGACUAUGCAGGCUCGUAUCCGUACGACGUUCCUGAUUAUGCGGGAUCUGGCGUUUUUACUCUGGAGGACUUCGUGGGGGACUGGCGUCAAACUGCUGGCUACAAUUUGGACCAGGUCUUAGAACAAGGAGGGGUGUCGUCGUUAUUCCAAAAUCUUGGAGUGAGCGUUACCCCAAUCCAGCGGAUAGUCCUCAGCGGCGAAAAUGGCCUUAAAAUAGACAUUCAUGUAAUAAUACCUUAUGAGGGGUUGUCUGGCGAUCAGAUGGGCCAAAUCGAGAAGAUAUUUAAAGUGGUUUAUCCUGUUGACGAUCACCACUUUAAGGUGAUCCUCCACUACGGAACACUCGUGAUCGACGGUGUAACUCCCAAUAUGAUUGACUAUUUUGGCCGCCCCUAUGAAGGGAUCGCCGUGUUCGACGGGAAAAAGAUAACAGUGACUGGAACACUGUGGAACGGGAACAAAAUUAUCGACGAAAGGCUAAUAAACCCAGACGGAUCGUUACUCUUUAGAGUCACGAUCAACGGGGUAACGGGGUGGCGAUUAUGUGAACGUAUUCUGGCUUGA
MFE Structure matches ribotree MFE Structure: True
.........
Ribotree-calculated DegScore: 213.20200000000008
Calculated DegScore: 213.20200000000006
.........
Ribotree-calculated dG(MFE): -111.56
Calculated dG(MFE): -111.56
.........
Ribotree-calculated CAI: 0.6940789108874253
Calculated CAI: 0.6940789108874253
.........
Ribotree-calculat

In [ ]:
# optional, if they want to ouput the double checked dataframe
# checked_data.to_csv('checked_data_filename')